In [1]:
from bs4 import BeautifulSoup
import requests
from IPython.core.display import display, HTML
from selenium import webdriver
import time
import os
import numpy as np
import pickle
import pandas as pd
import sys
import re
import seaborn as sns
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
%matplotlib inline 

In [24]:
#Launch driver
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)

In [3]:
#We want to scrape all the product pages for all results from backcountry.com's alpine touring skis

In [4]:
#First define function for loading up a url and getting soup object to play with
def load_soup(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html5lib')
    return soup


In [5]:
#Let's load up the results page for backcounty.com's alpine touring ski selection
url = 'https://www.backcountry.com/alpine-touring-skis'
url2 = 'https://www.backcountry.com/alpine-skis'
#soup = load_soup(url)
#soup

In [6]:
#Get all results pages, given start page
#Take first page url, return list of that page and all following as soup objects
def get_all_sub_results_pages(url):
    soup = load_soup(url)
    next_button = soup.find(class_= 'pag-next')
    if next_button != None:
        time.sleep(2)
        return [soup] + get_all_sub_results_pages('https://www.backcountry.com' + next_button.contents[0].get('href'))
    return [soup]


In [8]:
#Let's get all the results for alpine-touring-skis!
results_url = 'https://www.backcountry.com/alpine-touring-skis'
results_url2 = 'https://www.backcountry.com/alpine-skis'
all_results = get_all_sub_results_pages(results_url)
all_results2 = get_all_sub_results_pages(results_url2)
print(type(all_results))
print(len(all_results))
print(type(all_results[0]))
print(type(all_results2))
print(len(all_results2))
print(type(all_results2[0]))

<class 'list'>
3
<class 'bs4.BeautifulSoup'>
<class 'list'>
6
<class 'bs4.BeautifulSoup'>


In [9]:
#For scraping to get soup from a single product page, since it won't load straight from response text
#Must have driver running and called driver
def get_product_page_soup(url):
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html5lib')
    return soup

In [10]:
#Get list of all product page soups from list of results page soup objects
def get_all_product_soups(results_list):
    product_page_soups = []
    for results_page in results_list:
        product_url_list = ['https://www.backcountry.com' + soupob.contents[0].get('href') for soupob in results_page.find_all('div', class_='ui-pli-wrap js-pli-wrap')]
        for url in product_url_list:
            product_page_soups.append(get_product_page_soup(url))
            time.sleep(2)
    return product_page_soups

In [11]:
all_product_soups = get_all_product_soups(all_results)
all_product_soups2 = get_all_product_soups(all_results2)

In [36]:
all_product_soups2[13]

<!DOCTYPE html>
<html class="titlemetadescriptionpdp no-optimizelycustomizationtool rendermediaplaceholder no-newsacfooter no-cartitemsavings no-showmorecategories communityapiproductwall communityproductwall no-solrautorestart pageviewtrackingdtm no-closeoutcatalog no-migrationdroplet-bikebuilder no-listservicebackup recaptchav2 targeter_plp_centermiddle1 fullpricefeed fep_prefetchdns migrationdroplet-shopallbrands cartcookie tagdisablecustora no-footeremailsubs pricesyncvalidator externalcachedroplet-home no-activityfeed intllocation no-fep_async_css fblikeonreviews nativeappcheckout cartcheckoutv3 no-carouselarrows no-usearrows no-newcontactformheader no-sacglobalnavtest no-externalcachedroplet-collection lazyloadmobhhr filterformscalls migrationdroplet-productshighlights headerv3 singlepageaccount no-communitycategoryexperts bcckrakencleanurl dollardonation edgecontrolheaders migrationdroplet-categorypagebody no-advancedbikefitcalculator no-migrationdroplet-searchsortby no-canadast

In [13]:
print(len(all_product_soups))
print(type(all_product_soups[0]))
#display(HTML(all_product_soups[35].prettify()))

103
<class 'bs4.BeautifulSoup'>


In [81]:
#Take soup for one product page and return list of tuples with tech spec and value as strings(except length is list of ints)
def get_techspecs(product_page_soup):
    techspec_rows = product_page_soup.find_all('div', class_="tr ui-product-details__techspec-row")
    #if techspec_rows == None:
     #   techspec_rows = product_page_soup.find_all('div', class_='tr js-techspec-row"')
    techspec_list = []
    #print(len(product_page_soup))
    price = get_price(product_page_soup)
    techspec_list.append(('Price', price))
    size_list = get_available_sizes(product_page_soup)
    for row in techspec_rows :
        values = row.contents[1].text
#       values_list = re.split(r',', values_string)
        spec = row.contents[0].text
        if spec == 'Length' :
            size_strings = re.split(r',', values)
            size_strings_fix = [size for size in size_strings if size != '']
            size_floats = [float(re.sub(r'[^\d]','',string)) for string in size_strings_fix]
            values = [size for size in size_floats if size in size_list]
        techspec_list.append((spec, values))
    #print(techspec_list)
    return techspec_list

In [41]:
#Define function for actual price to consumer as string including $
def get_price(product_page_soup):
    price = product_page_soup.find('span', class_='product-pricing__sale js-product-pricing__sale')
    if price == None :
        price = product_page_soup.find('span', class_='product-pricing__sale')
    if price == None :    
        price = product_page_soup.find('span', class_='product-pricing__retail')
    return price.text

In [72]:
#return sizes as a list of floast -- all units are cm
def get_available_sizes(product_page_soup):
    dropdown_box = product_page_soup.find('ul', class_='buybox-dropdown__options js-basedropdown__options')
    #print(dropdown_box)
    size_list = [child.text for child in dropdown_box.contents]
    size_list_floats = [float(re.sub(r'[^\d]','',string)) for string in size_list]
    return size_list_floats

In [17]:
#Let's create a dictionary of feature keys to values that we can add to a list to create a data frame
  
def create_product_dict(techspec_list, feature_list):    
    product_dict = {}
    for feature in feature_list:
        for spec_tuple in techspec_list:
            if spec_tuple[0] == feature:
                product_dict[feature] = spec_tuple[1]
    return product_dict

In [69]:
#Function for turning products with multiple lengths into separate products
def expand_multiple_lengths(product_dict):
    dict_list = []
    #print(product_dict.keys())
    try :
        if len(product_dict['Length']) > 1 :
            for length in product_dict['Length']:
                new_dict = {}
                new_dict['Length'] = length
                other_features = [*product_dict]
                other_features.remove('Length')
                for feature in other_features:
                    feature_by_length = length_dictify(product_dict[feature])
                    if feature_by_length :
                        try : 
                            new_dict[feature] = feature_by_length[length]         
                        except :
                            new_dict[feature] = feature_by_length                    
                    new_dict[feature] = product_dict[feature]
                dict_list.append(new_dict)       
        else:
            new_dict = product_dict
            try:
                new_dict['Length'] = new_dict['Length'][0]
            except:
                new_dict['Length'] = None       
            dict_list.append(new_dict)
        return dict_list
    except:
        dict_list.append(product_dict)
        return(dict_list)


In [19]:
#Helper function matching value to length for string with mutiple lengths
def length_dictify(string):
    if type(string) == str:
        string += '['
        tuples = re.findall(r'(\d\d\d)cm.*?(\d.+?)[\[\(]', string)
        tuples = [(int(tuple_[0]), tuple_[1]) for tuple_ in tuples]
    else:
        return string
    return dict(tuples)

In [1]:
#Function for pulling out the value for a given length, otherwise take what's given
def choose_values_for_length(product_dict):
    return_dict = {}
    for key in product_dict:
        dictified = length_dictify(product_dict[key])
        if not dictified or type(dictified) != dict :
            return_dict[key] = product_dict[key]
        else:
            try :
                return_dict[key] = dictified[product_dict['Length']]
            except :
                return_dict[key] = dictified[list(dictified.keys())[0]]
    return return_dict
            

In [61]:
#Takes a list of product page soups and desired features and returns a data frame with the corresponding data
def create_data_frame(all_product_soups, feature_list):
    dict_list = []
    dict_list2 = []
    for product_soup in all_product_soups :
        techspec_list = get_techspecs(product_soup)
        product_dict = create_product_dict(techspec_list, feature_list)
        dict_list.extend(expand_multiple_lengths(product_dict))
    for dict_ in dict_list:
        dict_list2.append(choose_values_for_length(dict_))
    return pd.DataFrame(dict_list2)

In [22]:
#test_soup_1 = get_product_page_soup('https://www.backcountry.com/g3-findr-102-ski?skid=GGG002X-ONECOL-S174CM&ti=UExQIENhdDpBbHBpbmUgVG91cmluZyBTa2lzOjE6MTY6YmMtYWxwaW5lLXRvdXJpbmctc2tpcw==')
#test_soup_2 = get_product_page_soup('https://www.backcountry.com/dynafit-speed-90-ski?skid=DNF00CH-CAC-S167CM&ti=UExQIENhdDpBbHBpbmUgVG91cmluZyBTa2lzOjE6Mjg6YmMtYWxwaW5lLXRvdXJpbmctc2tpcw==')
#test_soups = [test_soup_1, test_soup_2]
#dftest = create_data_frame(test_soups, feature_list)
#dftest

In [85]:
feature_list = ['Length', 'Dimensions', 'Turn Radius', 'Claimed Weight', 'Price', 'Core']

df1 = create_data_frame(all_product_soups, feature_list)
df2 = create_data_frame(all_product_soups2, feature_list)

In [86]:
df1.shape

(175, 6)

In [87]:
df2.shape

(367, 6)

In [88]:
df = pd.concat([df1, df2])

In [89]:
df.shape

(542, 6)

In [90]:
#Pickle dataframe
df.to_pickle('ski_pickle3.pkl')

In [91]:
driver.close()